# TESTING A DIFFERENT SOLVER

In [1]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

In [2]:
import pandas as pd

cost = pd.read_csv("cost_vector.csv", index_col= 0 )
ebitda = pd.read_csv("ebitda_matrix.csv",index_col= 0)


In [3]:
import pyomo.environ as pyo

In [6]:
model = pyo.ConcreteModel()

num_y = cost.size

num_rows = ebitda.shape[0]
num_cols = ebitda.shape[1]

# Create x
model.rows = pyo.RangeSet(1,num_rows)
model.cols = pyo.RangeSet(1,num_cols)

# Define decision variables
model.x = pyo.Var(model.rows, model.cols, within = pyo.Binary)
model.y = pyo.Var(range(1, num_y + 1), within = pyo.Binary)

# Define the objective function programmatically
def profit_function(model):
    # Calculate total cost
    cost_total = sum(model.y[j] * cost.iloc[j - 1, 0] for j in range(1, num_y + 1))

    # Calculate total EBITDA
    ebitda_total = sum(model.y[j] * ebitda.iloc[i - 1, j - 1] * model.x[i, j] for i in model.rows for j in range(1, num_y + 1))

    # Calculate profit
    profit_total = ebitda_total - cost_total
    return profit_total

model.obj = pyo.Objective(rule = profit_function, sense = pyo.maximize)

# Define constraints
model.constraints = pyo.ConstraintList()
for i in range(1,num_rows):
    model.constraints.add(sum(model.x[i,j] for j in range(1,num_cols)) <= 1)


In [ ]:

solver = pyo.SolverFactory('bonmin')
# solver = pyo.SolverFactory('couenne') #one of these two algorithms
results = solver.solve(model, tee = True)

Bonmin 1.8.8 using Cbc 2.10.10 and Ipopt 3.13.2
bonmin: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale s

In [20]:
ebitda.size + cost.size

113025